<a href="https://colab.research.google.com/github/gazal-gupta/Deep_Learning/blob/master/Using_External_Embedings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from keras.layers import LSTM,GRU, Activation, Dense, Dropout,Flatten, Input, Embedding
from keras.models import Model

from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D , GlobalMaxPooling1D

In [2]:
#download embedings
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2019-12-29 04:55:14--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2019-12-29 04:55:19--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2019-12-29 04:55:19--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [3]:
#unzip the file 
!unzip glove*.zip

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
#store 300 size vector representation different words from the file to a disctionary
embedding_index = {}
f = open('glove.42B.300d.txt',encoding='utf-8')
for line in f:
  value = line.split()
  word = value[0]
  coeffs = np.asarray(value[1:],dtype = 'float32')
  embedding_index[word] = coeffs
f.close()

In [5]:
len(embedding_index)

1917494

In [6]:
# download data file to colab
!wget https://www.dropbox.com/s/1kvm3qqtxn5bhgx/yelp_labelled.txt

--2019-12-29 05:13:30--  https://www.dropbox.com/s/1kvm3qqtxn5bhgx/yelp_labelled.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/1kvm3qqtxn5bhgx/yelp_labelled.txt [following]
--2019-12-29 05:13:31--  https://www.dropbox.com/s/raw/1kvm3qqtxn5bhgx/yelp_labelled.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc3aa1550ea9d05e85ff1e9e948.dl.dropboxusercontent.com/cd/0/inline/AvJJXozYPsY91C1W3xKI6phu6SaIHLkB1QVQZVxZMfJmNlMeHdI_WE2BVU7CizUurYDN9Ub7g4xGS4xh8m5MCK14S6XEVYwnRXvhpODZh_b2Cw/file# [following]
--2019-12-29 05:13:31--  https://ucc3aa1550ea9d05e85ff1e9e948.dl.dropboxusercontent.com/cd/0/inline/AvJJXozYPsY91C1W3xKI6phu6SaIHLkB1QVQZVxZMfJmNlMeHdI_WE2BVU7CizUurYDN9Ub7g4xGS4xh8m5MCK14S6XEVYwnRXvhpODZh_b2Cw/file
Reso

In [0]:
data=pd.read_csv('/content/yelp_labelled.txt',sep='\t',header=None)

In [0]:
data.columns=['review','sentiment']

In [47]:
data.shape

(1000, 2)

In [48]:
data.head()

,review,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [0]:
from sklearn.model_selection import train_test_split
data_train,data_test=train_test_split(data,test_size=0.2)

In [0]:
x_train = data_train['review']
y_train = data_train['sentiment']
x_test  = data_test['review']
y_test  = data_test['sentiment']

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM,Activation,Dense,Input,Embedding,Dropout
from keras.models import Model

In [0]:
from nltk import word_tokenize

In [53]:
>>> import nltk
>>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
sent_lens=[len(word_tokenize(x)) for x in x_train]

In [55]:
max(sent_lens)

34

In [56]:
np.percentile(sent_lens,95)

25.0

training only on x_train, this will increase the acurracy only on test set. will help in project

In [0]:
max_len=25

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [58]:
seq_train_matrix.shape

(800, 25)

In [59]:
seq_train_matrix[2]

array([  0,   0,   1,  17,   4,   5, 134, 135, 500,  28,  27, 248,  89,
       250, 149, 251,  24,   1,  12,   4, 136,  11,   5, 135, 501],
      dtype=int32)

In [60]:
vocab_size

1823

In [0]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index 
# that we downloaded 

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
        
# if there are specific words which are not present in pretrained embedding 
# their weights will remain 0. if there are too many such words 
# then you should probably not use pretrained embeddings 


In [62]:
embedding_matrix.shape

(1824, 300)

play with the code by combining anyone with other dense code

dense2 = Dense(20,activation='relu')('dense1)
drop - Dropout(0.2)(dense2)

In [0]:
# Now while creating the model for text data , we'll keep embedding layer weights 
# non-trainable as we are supplying those values from our end 

inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer=LSTM(50)(embed)
dense1=Dense(10,activation='relu')(lstm_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [0]:
from keras.layers import Input, concatenate
# Cocanting with 2 LSTM model 

inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1=LSTM(50)(embed)
lstm_layer2=LSTM(100)(embed)
dense1=Dense(10,activation='relu')(lstm_layer1)
dense2=Dense(20,activation='relu')(lstm_layer2)
concat_layer = concatenate([dense1,dense2])
dense3 = Dense(5,activation='relu')(concat_layer)
drop=Dropout(0.2)(dense3)
final_layer=Dense(1,activation='sigmoid')(drop)

model=Model(inputs=inputs,outputs=final_layer)

In [65]:
# you can see that bulk of the weights come from embedding layer
# using pretrained weights for the same , brings down number of trainable parameters 
# by a huge amount. This in turn results in pretty short training times 
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 25, 300)      547200      text_input[0][0]                 
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 50)           70200       embedding_8[0][0]                
__________________________________________________________________________________________________
lstm_6 (LSTM)                   (None, 100)          160400      embedding_8[0][0]                
____________________________________________________________________________________________

25*300
50 LSTM as 1 output therfore 50 output
dense 10 and then dense 10

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [67]:
model.fit(seq_train_matrix,y_train,validation_data=[seq_test_matrix,y_test],epochs=10,class_weight={0:1,1:12},
          batch_size=10000)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 4s 5ms/step - loss: 5.2453 - acc: 0.4988 - val_loss: 0.6927 - val_acc: 0.5050
Epoch 2/10
800/800 [==============================] - 1s 1ms/step - loss: 4.6347 - acc: 0.4875 - val_loss: 0.6938 - val_acc: 0.4550
Epoch 3/10
800/800 [==============================] - 1s 1ms/step - loss: 4.5366 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.4800
Epoch 4/10
800/800 [==============================] - 1s 1ms/step - loss: 4.5276 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.4850
Epoch 5/10
800/800 [==============================] - 1s 1ms/step - loss: 4.5254 - acc: 0.4975 - val_loss: 0.6931 - val_acc: 0.4750
Epoch 6/10
800/800 [==============================] - 1s 1ms/step - loss: 4.5192 - acc: 0.5075 - val_loss: 0.6932 - val_acc: 0.4700
Epoch 7/10
800/800 [==============================] - 1s 1ms/step - loss: 4.5153 - acc: 0.5050 - val_loss: 0.6932 - val_acc: 0.4800
Epoch 8/10
800/800 [==========

In [0]:
p=model.predict(seq_test_matrix)

In [69]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.5002502252026824

In [70]:
inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
GRU_layer=GRU(50)(embed)
dense1=Dense(10,activation='relu')(GRU_layer)
drop=Dropout(0.2)(dense1)
final_layer=Dense(1,activation='sigmoid')(drop)

model_GRU=Model(inputs=inputs,outputs=final_layer)
model_GRU.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 25)                0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 25, 300)           547200    
_________________________________________________________________
gru_2 (GRU)                  (None, 50)                52650     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                510       
_________________________________________________________________
dropout_7 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 600,371
Trainable params: 53,171
Non-trainable params: 547,200
________________________________________________

In [0]:
model_GRU.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [72]:
model_GRU.fit(seq_train_matrix,y_train,validation_data=[seq_test_matrix,y_test],epochs=10,class_weight={0:1,1:12},
          batch_size=10000)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 2s 3ms/step - loss: 5.0663 - acc: 0.4775 - val_loss: 0.6930 - val_acc: 0.5150
Epoch 2/10
800/800 [==============================] - 0s 385us/step - loss: 4.6928 - acc: 0.5238 - val_loss: 0.6912 - val_acc: 0.5350
Epoch 3/10
800/800 [==============================] - 0s 378us/step - loss: 4.4901 - acc: 0.5437 - val_loss: 0.6908 - val_acc: 0.5400
Epoch 4/10
800/800 [==============================] - 0s 374us/step - loss: 4.3768 - acc: 0.5350 - val_loss: 0.6901 - val_acc: 0.5450
Epoch 5/10
800/800 [==============================] - 0s 376us/step - loss: 4.2033 - acc: 0.5300 - val_loss: 0.6902 - val_acc: 0.5300
Epoch 6/10
800/800 [==============================] - 0s 374us/step - loss: 4.0164 - acc: 0.5163 - val_loss: 0.6925 - val_acc: 0.5050
Epoch 7/10
800/800 [==============================] - 0s 372us/step - loss: 3.8021 - acc: 0.5200 - val_loss: 0.6987 - val_acc: 0.4900
Epoch 8/10
800/800

In [80]:
p=model_GRU.predict(seq_test_matrix)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.6168551696526874

Analysis by Conv1D

In [78]:
#model.add(Conv1D (kernel_size = (200), filters = 20, batch_input_shape=input_shape, activation='elu'))
from keras.layers import GlobalMaxPooling1D
model_3 = Sequential()
model_3.add(Embedding(vocab_size+1,300,input_length=max_len,weights=[embedding_matrix],trainable=False))
model_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', padding = 'same'))
model_3.add(GlobalMaxPooling1D())
#model_3.add(Flatten())
model_3.add(Dense(150, activation='relu'))
model_3.add(Dropout(0.2))
model_3.add(Dense(1,activation='sigmoid'))
# Compile model
model_3.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])
model_3.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 25, 300)           547200    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 25, 64)            38464     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 150)               9750      
_________________________________________________________________
dropout_9 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 151       
Total params: 595,565
Trainable params: 48,365
Non-trainable params: 547,200
___________________________________________

In [79]:
model_3.fit(seq_train_matrix,y_train, validation_data=(seq_test_matrix,y_test),batch_size=50,epochs=50, verbose=1)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 2s 2ms/step - loss: 0.6776 - acc: 0.5700 - val_loss: 0.6331 - val_acc: 0.6750
Epoch 2/50
800/800 [==============================] - 0s 225us/step - loss: 0.5216 - acc: 0.7987 - val_loss: 0.5406 - val_acc: 0.7200
Epoch 3/50
800/800 [==============================] - 0s 257us/step - loss: 0.3594 - acc: 0.8725 - val_loss: 0.4905 - val_acc: 0.7700
Epoch 4/50
800/800 [==============================] - 0s 226us/step - loss: 0.2393 - acc: 0.9200 - val_loss: 0.4613 - val_acc: 0.7700
Epoch 5/50
800/800 [==============================] - 0s 221us/step - loss: 0.1626 - acc: 0.9488 - val_loss: 0.4533 - val_acc: 0.7900
Epoch 6/50
800/800 [==============================] - 0s 222us/step - loss: 0.1080 - acc: 0.9713 - val_loss: 0.4666 - val_acc: 0.8000
Epoch 7/50
800/800 [==============================] - 0s 227us/step - loss: 0.0698 - acc: 0.9925 - val_loss: 0.4948 - val_acc: 0.7900
Epoch 8/50
800/800

In [81]:
p=model_3.predict(seq_test_matrix)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,p)

0.8765889300370333